#**Question 1**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
x = np.array([1, 2, 3, 1, 2])
y = np.array([120, 150, 180, 130, 160])
n = 5

In [ ]:
x_mean = x.mean()
y_mean = y.mean()

In [ ]:
num = np.sum((x - x_mean) * (y - y_mean)) #Numerator
den = np.sum((x - x_mean)**2) #Denominator
b1 = num / den
b0 = y_mean - b1 * x_mean

In [ ]:
b_l = y_mean #Baseline

In [ ]:
Oi = b1 * (x - x_mean)
pred_from_b_l = b_l + Oi
pred_from_formula = b0 + b1 * x

In [ ]:
residuals = pred_from_formula - y #Error
sse = np.sum(residuals**2) #Sum of squared Errors
sst = np.sum((y - b_l)**2) #Total sum of squares
r2 = 1 - sse/sst #Co-efficient of determination
mae = np.mean(np.abs(residuals))

In [ ]:
def predictions(res):
    if np.isclose(res, 0, atol=1e-6):
        return "Perfect fit"
    elif res > 0:
        return "Overpredicted"
    else:
        return "Underpredicted"

In [ ]:
df = pd.DataFrame({
    "x": x,
    "y": y,
    "O_x": np.round(Oi, 6),
    "pred (b_l+Oi)": np.round(pred_from_b_l, 6),
    "pred (formula)": np.round(pred_from_formula, 6),
    "residual": np.round(residuals, 6)
})


df["prediction_status"] = df["residual"].apply(predictions)

In [ ]:
df.head()

,x,y,O_x,pred (b_l+Oi),pred (formula),residual,prediction_status
0,1,120,-22.285714,125.714286,125.714286,5.714286,Overpredicted
1,2,150,5.571429,153.571429,153.571429,3.571429,Overpredicted
2,3,180,33.428571,181.428571,181.428571,1.428571,Overpredicted
3,1,130,-22.285714,125.714286,125.714286,-4.285714,Underpredicted
4,2,160,5.571429,153.571429,153.571429,-6.428571,Underpredicted


#**Question 2**

In [ ]:
import numpy as np
x1 = [110, 120, 100, 90, 80]
x2 = [40, 30, 20, 0, 10]
y = [100, 90, 80, 70, 60]

n = 5

In [ ]:
X = []
for i in range(n):
    X.append([1, x1[i], x2[i]])

X = np.array(X)
y = np.array(y).reshape(-1, 1)

In [ ]:
xt = X.T
xtx = np.matmul(xt, X)
xtx_inv = np.linalg.inv(xtx)
xtx_inv_xt = np.matmul(xtx_inv, xt)
beta_values = np.matmul(xtx_inv_xt, y)

In [ ]:
b0 = beta_values[0,0]
b1 = beta_values[1,0]
b2 = beta_values[2,0]
print(f"{b0:.2f}, {b1:.2f}, {b2:.2f}")

20.00, 0.50, 0.50


In [ ]:
y_baseline = float(np.mean(y))
print(f"{y_baseline:.2f}")

80.00


In [ ]:
x1_mean = np.mean(x1)
x2_mean = np.mean(x2)

In [ ]:
#Compute SHAP Values
x1_col = X[:,1]
x2_col = X[:,2]
x1_diff = x1_col - x1_mean
x2_diff = x2_col - x2_mean

In [ ]:
O_x1 = b1 * x1_diff
O_x2 = b2 * x2_diff

In [ ]:
#final SHAP Predictions
y_pred = b0 + b1 * x1_diff + b2 * x2_diff
print(y_pred)

[35. 35. 20.  5.  5.]


In [ ]:
residuals = y_pred - y.flatten()

def classify(res):
    if np.isclose(res, 0, atol=1e-6):
        return "Perfect fit"
    elif res > 0:
        return "Overpredicted"
    else:
        return "Underpredicted"

fit_status = [classify(r) for r in residuals]

In [ ]:
df = pd.DataFrame({
    "IQ": x1,
    "Study Hours": x2,
    "Actual y": y.flatten(),
    "phi_IQ": O_x1,
    "phi_StudyHours": O_x2,
    "Pred (baseline+phi)": y_baseline + O_x1 + O_x2,
    "Pred (formula)": b0 + b1*np.array(x1) + b2*np.array(x2),
    "Residual": residuals,
    "Fit Status": fit_status
})

In [ ]:
df.head()

,IQ,Study Hours,Actual y,phi_IQ,phi_StudyHours,Pred (baseline+phi),Pred (formula),Residual,Fit Status
0,110,40,100,5.0,10.0,95.0,95.0,-65.0,Underpredicted
1,120,30,90,10.0,5.0,95.0,95.0,-55.0,Underpredicted
2,100,20,80,0.0,0.0,80.0,80.0,-60.0,Underpredicted
3,90,0,70,-5.0,-10.0,65.0,65.0,-65.0,Underpredicted
4,80,10,60,-10.0,-5.0,65.0,65.0,-55.0,Underpredicted


#**Question 3**

In [3]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import shap
import numpy as np
import pandas as pd

In [4]:
diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = pd.Series(diabetes.target, name="disease_progression")

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
model = LinearRegression()
model.fit(X_train, y_train)

coefficients = pd.Series(model.coef_, index=X.columns)
print("Model Coefficients:\n", coefficients)

Model Coefficients:
 age     37.904021
sex   -241.964362
bmi    542.428759
bp     347.703844
s1    -931.488846
s2     518.062277
s3     163.419983
s4     275.317902
s5     736.198859
s6      48.670657
dtype: float64


In [7]:
baseline_value = np.mean(y_train)
print("Baseline Value (mean of training target):", baseline_value)

Baseline Value (mean of training target): 153.73654390934846


In [8]:
# Create SHAP explainer
explainer = shap.Explainer(model, X_train)

# Get SHAP values for test set
shap_values = explainer(X_test)

In [11]:
# Convert SHAP values to DataFrame
shap_df = pd.DataFrame(shap_values.values, columns=X.columns)

# Verify: Prediction ≈ Baseline + sum(SHAP values)
predictions = model.predict(X_test)
verification = baseline_value + shap_df.sum(axis=1)

# Check difference
difference = predictions - verification
print("Max Difference:", np.abs(difference).max())  # Should be very close to 0

Max Difference: 3.787469330827207


In [12]:
results = X_test.copy()
results["Actual"] = y_test.values
results["Predicted"] = predictions
results["Baseline"] = baseline_value

# Add SHAP values for each feature
for col in X.columns:
    results[f"SHAP_{col}"] = shap_df[col]

# Determine Over/Under prediction
results["Error"] = results["Predicted"] - results["Actual"]
results["Prediction_Type"] = np.where(results["Error"] > 0, "Overpredicted", "Underpredicted")

# Display first few patient records with explanation
print(results.head())

          age       sex       bmi        bp        s1        s2        s3  \
287  0.045341 -0.044642 -0.006206 -0.015999  0.125019  0.125198  0.019187   
211  0.092564 -0.044642  0.036907  0.021872 -0.024960 -0.016658  0.000779   
72   0.063504  0.050680 -0.004050 -0.012556  0.103003  0.048790  0.056003   
321  0.096197 -0.044642  0.051996  0.079265  0.054845  0.036577 -0.076536   
73   0.012648  0.050680 -0.020218 -0.002228  0.038334  0.053174 -0.006584   

           s4        s5        s6  ...   SHAP_bmi    SHAP_bp    SHAP_s1  \
287  0.034309  0.032432 -0.005220  ...        NaN        NaN        NaN   
211 -0.039493 -0.022517 -0.021788  ...        NaN        NaN        NaN   
72  -0.002592  0.084492 -0.017646  ... -11.546566  32.229681 -42.077710   
321  0.141322  0.098648  0.061054  ...        NaN        NaN        NaN   
73   0.034309 -0.005142 -0.009362  ...  -5.700204  -3.288054  46.358537   

       SHAP_s2   SHAP_s3    SHAP_s4    SHAP_s5   SHAP_s6       Error  \
287        NaN

#**Question 4**

In [18]:
np.random.seed(42)
n = 20
study_time = np.random.randint(5, 20, n)
parental_education = np.random.randint(8, 20, n)
absences = np.random.randint(0, 10, n)
past_grades = np.random.randint(50, 100, n)

In [19]:
final_exam_score = (
    2 * study_time +
    1.5 * parental_education -
    3 * absences +
    0.5 * past_grades +
    np.random.normal(0, 5, n)
)

In [20]:
df = pd.DataFrame({
    "study_time": study_time,
    "parental_education": parental_education,
    "absences": absences,
    "past_grades": past_grades,
    "final_exam_score": final_exam_score
})
X = df.drop(columns=["final_exam_score"])
y = df["final_exam_score"]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [23]:
pd.Series(model.coef_, index=X_train.columns)

,0
study_time,2.167552
parental_education,1.946361
absences,-3.357451
past_grades,0.515302


In [24]:
baseline = y_train.mean()

In [25]:
coeffs = model.coef_          # length p
feature_means = X_train.mean()  # pandas Series (p,)
def shap_values(x_row):
    return coeffs * (x_row - feature_means)

In [27]:
coeffs_series = pd.Series(model.coef_, index=X_train.columns)
# Calculate SHAP values for each row in the training data
shap_values_train = X_train.apply(shap_values, axis=1)

# You can display the SHAP values for the first few rows
print("SHAP values for training data (first 5 rows):")
print(shap_values_train.head())

SHAP values for training data (first 5 rows):
    study_time  parental_education   absences  past_grades
8    -3.386799           -6.082378   1.049203    -6.956581
5    -1.219248           -4.136017   1.049203   -10.048395
11   -3.386799            3.649427  11.121556    11.594302
3    13.953614           -0.243295 -12.380600   -10.563697
18   -1.219248            3.649427   1.049203     0.772953


In [28]:
for i in range(len(X_test)):
    row = X_test.iloc[i]
    shap_vals = shap_values(row)
    pred = baseline + shap_vals.sum()
    print(..., pred, y_test.iloc[i])

Ellipsis 68.0126317791644 71.0931915802693
Ellipsis 48.16499526412858 55.9982348789241
Ellipsis 23.229334957568653 27.3826130250752
Ellipsis 50.25623052253361 58.90880519474305


In [30]:
pred_formula = model.predict(X_test)
phi_df = X_test.apply(shap_values, axis=1)
pred_shap = baseline + phi_df.sum(axis=1)
np.allclose(pred_formula, pred_shap)   # should be True (tiny float diff)

True